# Data cleaning of art auction dataset

## Import packages and data

In [514]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import warnings
import xlrd
warnings.filterwarnings('ignore')

In [515]:
df_christies = pd.read_excel('Data_raw/HK_Christies_2016-2020.xls')
df_sothebys = pd.read_excel('Data_raw/HK_Sothebys_2016-2020.xls')
df_artbank = pd.read_csv('Data_raw/cs_stony_brook.csv')

In [516]:
# transform the data into a dataframe
df_christies = pd.DataFrame(df_christies)
df_sothebys = pd.DataFrame(df_sothebys)
df_artbank = pd.DataFrame(df_artbank)

# Lets first clean the christies dataset

### Columns as lower case

In [517]:
# all column names as lowercase
df_christies.columns = df_christies.columns.str.lower()

# rename sales price column
df_christies.rename(columns={'Sales_Price_Dollar':'sales_price'},inplace=True)

In [518]:
df_christies.head()

,artist,title,sales_price_dollar,low_estimate,high_estimate,signature,size,edition,foundry,created,medium_clean,medium,auction_lot,auction_date
0,Binhong Huang,Misty Landscape,45900.0,38490.0,64150.0,Inscribed and Signed,"33.66"" x 16.93"" ...",NaN,NaN,1947,Works on Paper,Watercolor and ink / Paper,1364,05/30/2017
1,Binhong Huang,Conversations in the Mountain,NaN,38490.0,64150.0,Inscribed and Signed,"13.58"" x 10.43"" ...",NaN,NaN,1990,Works on Paper,Watercolor and ink / Paper,1365,05/30/2017
2,Yun Tang,Peony and Insects / Peony and Butterfly (2),22950.0,15400.0,23090.0,Inscribed and Signed,NaN,NaN,NaN,NaN,Works on Paper,Watercolor and ink / Paper,1366,05/30/2017
3,Binhong Huang,Retreat in the Mountains,433330.0,320750.0,449050.0,Inscribed and Signed,"40.94"" x 15.55"" ...",NaN,NaN,NaN,Works on Paper,Watercolor and ink / Paper,1367,05/30/2017
4,Yun Tang,Villagers in the Woods,12240.0,10260.0,12830.0,Inscribed and Signed,"39.53"" x 18.82"" ...",NaN,NaN,1940,Works on Paper,Watercolor and ink / Paper,1368,05/30/2017


### Separate size into height and width

In [519]:
# split size column into height and width
df_christies['height'] = df_christies['size'].str.split('"').str[0]
df_christies['width'] = df_christies['size'].str.split('x').str[1].str.split('"').str[0]

# remove white space
df_christies['height'] = df_christies['height'].str.strip()
df_christies['width'] = df_christies['width'].str.strip()

# convert height and width to float
df_christies['height'] = df_christies['height'].astype(float)
df_christies['width'] = df_christies['width'].astype(float)

# drop size column
df_christies.drop(columns=['size'],inplace=True)

In [520]:
df_christies.head()

,artist,title,sales_price_dollar,low_estimate,high_estimate,signature,edition,foundry,created,medium_clean,medium,auction_lot,auction_date,height,width
0,Binhong Huang,Misty Landscape,45900.0,38490.0,64150.0,Inscribed and Signed,NaN,NaN,1947,Works on Paper,Watercolor and ink / Paper,1364,05/30/2017,33.66,16.93
1,Binhong Huang,Conversations in the Mountain,NaN,38490.0,64150.0,Inscribed and Signed,NaN,NaN,1990,Works on Paper,Watercolor and ink / Paper,1365,05/30/2017,13.58,10.43
2,Yun Tang,Peony and Insects / Peony and Butterfly (2),22950.0,15400.0,23090.0,Inscribed and Signed,NaN,NaN,NaN,Works on Paper,Watercolor and ink / Paper,1366,05/30/2017,NaN,NaN
3,Binhong Huang,Retreat in the Mountains,433330.0,320750.0,449050.0,Inscribed and Signed,NaN,NaN,NaN,Works on Paper,Watercolor and ink / Paper,1367,05/30/2017,40.94,15.55
4,Yun Tang,Villagers in the Woods,12240.0,10260.0,12830.0,Inscribed and Signed,NaN,NaN,1940,Works on Paper,Watercolor and ink / Paper,1368,05/30/2017,39.53,18.82


### Clean the created column

In [521]:
def clean(df):
    # remove all c. in created and keep only the year
    df['created'] = df['created'].str.replace('c.','')

    # remove all early, mid, e., 's in created and keep only the year
    df['created'] = df['created'].str.replace('early','')
    df['created'] = df['created'].str.replace('mid','')
    df['created'] = df['created'].str.replace('e.','')
    df['created'] = df['created'].str.replace("'s",'')

    # remove all white space
    df['created'] = df['created'].str.strip()

    # if there is an and in created, take int average of the two years
    df['created'] = df['created'].str.split('and').str[0]

    # if there are two years, keep the first one
    df['created'] = df['created'].str.split('-').str[0]

    # if / in created, keep the first one
    df['created'] = df['created'].str.split('/').str[0]

    # remove all letters in created and keep only the year
    df['created'] = df['created'].str.replace('[a-zA-Z]','')

    #  split created on . and keep the bigger number
    df['created'] = df['created'].str.split('.')

    # if there is a list, keep the bigger number
    df['created'] = df['created'].apply(lambda x: max(x) if type(x) == list else x)

    #  remove all white space
    df['created'] = df['created'].str.strip()

    # remove all non numeric values
    df['created'] = df['created'].str.replace('[^0-9]','')

    # if nan pass else convert to int
    df['created'] = df['created'].apply(lambda x: x if pd.isnull(x) else int(x))

    # if values below 99, add 1900
    df['created'] = df['created'].apply(lambda x: x+1900 if x < 99 else x)

    # convert created to string
    df['created'] = df['created'].astype(str)

    # split it on . and keep the first number
    df['created'] = df['created'].str.split('.').str[0]

    #  if not nan, convert created to timestamp (year)
    df['created'] = df_christies['created'].apply(lambda x: pd.to_datetime(x, format='%Y', errors='ignore') if pd.notnull(x) else x)



In [522]:
clean(df_christies)

In [523]:
df_christies.head()

,artist,title,sales_price_dollar,low_estimate,high_estimate,signature,edition,foundry,created,medium_clean,medium,auction_lot,auction_date,height,width
0,Binhong Huang,Misty Landscape,45900.0,38490.0,64150.0,Inscribed and Signed,NaN,NaN,1947-01-01 00:00:00,Works on Paper,Watercolor and ink / Paper,1364,05/30/2017,33.66,16.93
1,Binhong Huang,Conversations in the Mountain,NaN,38490.0,64150.0,Inscribed and Signed,NaN,NaN,1990-01-01 00:00:00,Works on Paper,Watercolor and ink / Paper,1365,05/30/2017,13.58,10.43
2,Yun Tang,Peony and Insects / Peony and Butterfly (2),22950.0,15400.0,23090.0,Inscribed and Signed,NaN,NaN,NaT,Works on Paper,Watercolor and ink / Paper,1366,05/30/2017,NaN,NaN
3,Binhong Huang,Retreat in the Mountains,433330.0,320750.0,449050.0,Inscribed and Signed,NaN,NaN,NaT,Works on Paper,Watercolor and ink / Paper,1367,05/30/2017,40.94,15.55
4,Yun Tang,Villagers in the Woods,12240.0,10260.0,12830.0,Inscribed and Signed,NaN,NaN,1940-01-01 00:00:00,Works on Paper,Watercolor and ink / Paper,1368,05/30/2017,39.53,18.82


do we want to keep it as timestamp or simply an integer ?

### Signature column

In [524]:
#  all signature to lowercase
df_christies['signature'] = df_christies['signature'].str.lower()

In [525]:
# print unique values in signature column
df_christies['signature'].unique()

array(['inscribed and signed', 'signed', "artist's seal", 'inscribed',
       nan, 'signed and dated', 'signed monogram', 'signed and titled',
       'signed lower right', 'signed lower left', 'signed on reverse',
       'unavailable', 'inscribed and not signed', 'by estate', 'stamped',
       'unsigned', 'signed on label', 'signed with certificate',
       'signed on stretcher'], dtype=object)

In [526]:
# create an empty signed column
df_christies['signed'] = np.nan

# create an empty stamped column
df_christies['stamped'] = np.nan

# create an empty inscribed column
df_christies['inscribed'] = np.nan

# create an empty dated column
df_christies['dated'] = np.nan

In [527]:
# create a signed column: if nan pass, if "not signed" or "unsigned" in signature, signed = 0, if "signed" in signature signed = 1, else signed = 0
def signed(df):
    for i in range(len(df)):
        if pd.isnull(df['signature'][i]):
            df['signed'][i] = np.nan
        elif 'not signed' in df['signature'][i] or 'unsigned' in df['signature'][i]:
            df['signed'][i] = 0
        elif 'signed' in df['signature'][i]:
            df['signed'][i] = 1
        else:
            df['signed'][i] = 0

In [528]:
#  same for stamped 
def stamped(df):
    for i in range(len(df)):
        if pd.isnull(df['signature'][i]):
            df['stamped'][i] = np.nan
        elif 'not stamped' in df['signature'][i] or 'unstamped' in df['signature'][i]:
            df['stamped'][i] = 0
        elif 'stamped' in df['signature'][i]:
            df['stamped'][i] = 1
        else:
            df['stamped'][i] = 0

In [529]:
def inscribed(df):
    for i in range(len(df)):
        if pd.isnull(df['signature'][i]):
            df['inscribed'][i] = np.nan
        elif 'not inscribed' in df['signature'][i] or 'uninscribed' in df['signature'][i]:
            df['inscribed'][i] = 0
        elif 'inscribed' in df['signature'][i]:
            df['inscribed'][i] = 1
        else:
            df['inscribed'][i] = 0

In [530]:
# same for dated
def dated(df):
    for i in range(len(df)):
        if pd.isnull(df['signature'][i]):
            df['dated'][i] = np.nan
        elif 'not dated' in df['signature'][i] or 'undated' in df['signature'][i]:
            df['dated'][i] = 0 
        elif 'dated' in df['signature'][i]:
            df['dated'][i] = 1
        else:
            df['dated'][i] = 0

In [531]:
signed(df_christies)
stamped(df_christies)
inscribed(df_christies)
dated(df_christies)

In [532]:
# drop signature column
df_christies.drop(columns=['signature'],inplace=True)

In [533]:
df_christies.head()

,artist,title,sales_price_dollar,low_estimate,high_estimate,edition,foundry,created,medium_clean,medium,auction_lot,auction_date,height,width,signed,stamped,inscribed,dated
0,Binhong Huang,Misty Landscape,45900.0,38490.0,64150.0,NaN,NaN,1947-01-01 00:00:00,Works on Paper,Watercolor and ink / Paper,1364,05/30/2017,33.66,16.93,1.0,0.0,1.0,0.0
1,Binhong Huang,Conversations in the Mountain,NaN,38490.0,64150.0,NaN,NaN,1990-01-01 00:00:00,Works on Paper,Watercolor and ink / Paper,1365,05/30/2017,13.58,10.43,1.0,0.0,1.0,0.0
2,Yun Tang,Peony and Insects / Peony and Butterfly (2),22950.0,15400.0,23090.0,NaN,NaN,NaT,Works on Paper,Watercolor and ink / Paper,1366,05/30/2017,NaN,NaN,1.0,0.0,1.0,0.0
3,Binhong Huang,Retreat in the Mountains,433330.0,320750.0,449050.0,NaN,NaN,NaT,Works on Paper,Watercolor and ink / Paper,1367,05/30/2017,40.94,15.55,1.0,0.0,1.0,0.0
4,Yun Tang,Villagers in the Woods,12240.0,10260.0,12830.0,NaN,NaN,1940-01-01 00:00:00,Works on Paper,Watercolor and ink / Paper,1368,05/30/2017,39.53,18.82,1.0,0.0,1.0,0.0


## Medium

In [534]:
# all medium_clean and medium to lowercase
df_christies['medium_clean'] = df_christies['medium_clean'].str.lower()
df_christies['medium'] = df_christies['medium'].str.lower()

In [535]:
# in medium_clean remove "works on"
df_christies['medium_clean'] = df_christies['medium_clean'].str.replace('works on','')

# remove all white space
df_christies['medium_clean'] = df_christies['medium_clean'].str.strip()

In [536]:
# print all unique editions
df_christies['medium_clean'].unique()

array(['paper', 'painting', 'mixed media', 'sculpture', 'other',
       'unknown', 'print'], dtype=object)

In [537]:
# print all unique medium
len(df_christies['medium'].unique())

847

In [538]:
#  cahnge name of medium_clean to category
df_christies.rename(columns={'medium_clean':'category'},inplace=True)

### Auction date

In [539]:
# auction_date as timestamp
df_christies['auction_date'] = pd.to_datetime(df_christies['auction_date'], format='%d/%m/%Y', errors='ignore')

### Auction location

In [540]:
# add auction location column with "Christie's Hong Kong"
df_christies['auction_location'] = 'Christie\'s Hong Kong'

In [569]:
#  quickly rename one column
df_christies.rename(columns={'sales_price_dollar':'sales_price'},inplace=True)

In [570]:
df_christies.head()

,artist,title,sales_price,low_estimate,high_estimate,edition,foundry,created,category,medium,auction_lot,auction_date,height,width,signed,stamped,inscribed,dated,auction_location
0,Binhong Huang,Misty Landscape,45900.0,38490.0,64150.0,NaN,NaN,1947-01-01 00:00:00,paper,watercolor and ink / paper,1364,05/30/2017,33.66,16.93,1.0,0.0,1.0,0.0,Christie's Hong Kong
1,Binhong Huang,Conversations in the Mountain,NaN,38490.0,64150.0,NaN,NaN,1990-01-01 00:00:00,paper,watercolor and ink / paper,1365,05/30/2017,13.58,10.43,1.0,0.0,1.0,0.0,Christie's Hong Kong
2,Yun Tang,Peony and Insects / Peony and Butterfly (2),22950.0,15400.0,23090.0,NaN,NaN,NaT,paper,watercolor and ink / paper,1366,05/30/2017,NaN,NaN,1.0,0.0,1.0,0.0,Christie's Hong Kong
3,Binhong Huang,Retreat in the Mountains,433330.0,320750.0,449050.0,NaN,NaN,NaT,paper,watercolor and ink / paper,1367,05/30/2017,40.94,15.55,1.0,0.0,1.0,0.0,Christie's Hong Kong
4,Yun Tang,Villagers in the Woods,12240.0,10260.0,12830.0,NaN,NaN,1940-01-01 00:00:00,paper,watercolor and ink / paper,1368,05/30/2017,39.53,18.82,1.0,0.0,1.0,0.0,Christie's Hong Kong


I think this is pretty good  ?

# Lets clean the sothebys dataset

In [542]:
df_sothebys = pd.read_excel('Data_raw/HK_Sothebys_2016-2020.xls')
df_sothebys = pd.DataFrame(df_sothebys)

### Rename/Remove certain columns

In [543]:
# column names in lowercase
df_sothebys.columns = df_sothebys.columns.str.lower()

In [544]:
# rename columns: low_estimate_dollar to low_estimate, high_estimate_dollar to high_estimate
df_sothebys.rename(columns={'low_estimate_dollar':'low_estimate','high_estimate_dollar':'high_estimate', 'signed':'signature'}, inplace=True)

In [545]:
#  drop online_dummy and dummy_sold columns
df_sothebys.drop(columns=['online_dummy','dummy_sold'],inplace=True)

### Convert size to height and width

In [546]:
#  rename size_cm to size
df_sothebys.rename(columns={'size_cm':'size'},inplace=True)

In [547]:
# split size column into height and width
df_sothebys['height'] = df_sothebys['size'].str.split('cm').str[0]
df_sothebys['width'] = df_sothebys['size'].str.split('x').str[1].str.split('cm').str[0]

# remove white space
df_sothebys['height'] = df_sothebys['height'].str.strip()
df_sothebys['width'] = df_sothebys['width'].str.strip()

# convert height and width to float
df_sothebys['height'] = df_sothebys['height'].astype(float)
df_sothebys['width'] = df_sothebys['width'].astype(float)

# drop size column
df_sothebys.drop(columns=['size'],inplace=True)

In [583]:
# convert height and width from cm to inches
df_sothebys['height'] = round(df_sothebys['height'] / 2.54,2)
df_sothebys['width'] = round(df_sothebys['width'] / 2.54,2)

### Created year

In [548]:
# what is the type of created column
df_sothebys['created'].dtype

dtype('O')

In [549]:
def clean(df):
    # remove all c. in created and keep only the year
    df['created'] = df['created'].str.replace('c.','')

    # remove all early, mid, e., 's in created and keep only the year
    df['created'] = df['created'].str.replace('early','')
    df['created'] = df['created'].str.replace('mid','')
    df['created'] = df['created'].str.replace('e.','')
    df['created'] = df['created'].str.replace("'s",'')

    # remove all white space
    df['created'] = df['created'].str.strip()

    # if there is an and in created, take int average of the two years
    df['created'] = df['created'].str.split('and').str[0]

    # if there are two years, keep the first one
    df['created'] = df['created'].str.split('-').str[0]

    # if / in created, keep the first one
    df['created'] = df['created'].str.split('/').str[0]

    # remove all letters in created and keep only the year
    df['created'] = df['created'].str.replace('[a-zA-Z]','')

    #  split created on . and keep the bigger number
    df['created'] = df['created'].str.split('.')

    # if there is a list, keep the bigger number
    df['created'] = df['created'].apply(lambda x: max(x) if type(x) == list else x)

    #  remove all white space
    df['created'] = df['created'].str.strip()

    # remove all non numeric values
    df['created'] = df['created'].str.replace('[^0-9]','')

    # if created is '' replace with np.nan
    df['created'] = df['created'].replace('',np.nan)

    # if nan pass else convert to int
    df['created'] = df['created'].apply(lambda x: x if pd.isnull(x) else int(x))



In [550]:
#  find unique value in created column
clean(df_sothebys)

In [551]:
df_sothebys['created'] = df_sothebys['created'].apply(lambda x: x if x > 99 else x + 1900)

In [552]:
# convert created to string
df_sothebys['created'] = df_sothebys['created'].astype(str)

# split it on . and keep the first number
df_sothebys['created'] = df_sothebys['created'].str.split('.').str[0]

In [553]:
#  if not nan, convert created to timestamp (year)
df_sothebys['created'] = df_sothebys['created'].apply(lambda x: pd.to_datetime(x, format='%Y', errors='ignore') if pd.notnull(x) else x)

In [554]:
df_sothebys.head()

,artist,title,sales_dollar,low_estimate,high_estimate,signature,edition,foundry,created,category,medium,auction_lot,auction_date,auction_location,height,width
0,Liang Guan,STILL LIFE,240190,25620.0,51240.0,Signed,NaN,NaN,NaT,Paintings,Oil / Paperboard,1001,3/31/19,Sotheby's Hong Kong,28.0,33.0
1,Liang Guan,COMPOSITION,1252180,256200.0,512400.0,Unavailable,NaN,NaN,1948-01-01 00:00:00,Paintings,Oil / Canvas,1002,3/31/19,Sotheby's Hong Kong,54.6,64.3
2,Liang Guan,MONKEY KING MAKING HAVOC IN HEAVEN,1713340,256200.0,512400.0,Signed and Dated,NaN,NaN,1980-01-01 00:00:00,Works on Paper,Ink and colour / Paper,1003,3/31/19,Sotheby's Hong Kong,94.0,179.0
3,Liang Guan,COWHERD,112090,64050.0,102480.0,Signed and Dated,NaN,NaN,1977-01-01 00:00:00,Works on Paper,Ink and colour / Paper,1004,3/31/19,Sotheby's Hong Kong,98.0,90.5
4,Yen-Yung Ting,THE EIGHT IMMORTALS,NaN,102480.0,204960.0,Signed and Dated,NaN,NaN,1975-01-01 00:00:00,Works on Paper,Ink and colour / Paper,1005,3/31/19,Sotheby's Hong Kong,137.0,70.0


### Signature separation

In [555]:
# signature column to lowercase
df_sothebys['signature'] = df_sothebys['signature'].str.lower()

In [556]:
# create an empty signed column
df_sothebys['signed'] = np.nan

# create an empty stamped column
df_sothebys['stamped'] = np.nan

# create an empty inscribed column
df_sothebys['inscribed'] = np.nan

# create an empty dated column
df_sothebys['dated'] = np.nan

In [557]:
signed(df_sothebys)
stamped(df_sothebys)
inscribed(df_sothebys)
dated(df_sothebys)

In [558]:
# drop signature column
df_sothebys.drop(columns=['signature'],inplace=True)

In [559]:
df_sothebys.head()

,artist,title,sales_dollar,low_estimate,high_estimate,edition,foundry,created,category,medium,auction_lot,auction_date,auction_location,height,width,signed,stamped,inscribed,dated
0,Liang Guan,STILL LIFE,240190,25620.0,51240.0,NaN,NaN,NaT,Paintings,Oil / Paperboard,1001,3/31/19,Sotheby's Hong Kong,28.0,33.0,1.0,0.0,0.0,0.0
1,Liang Guan,COMPOSITION,1252180,256200.0,512400.0,NaN,NaN,1948-01-01 00:00:00,Paintings,Oil / Canvas,1002,3/31/19,Sotheby's Hong Kong,54.6,64.3,0.0,0.0,0.0,0.0
2,Liang Guan,MONKEY KING MAKING HAVOC IN HEAVEN,1713340,256200.0,512400.0,NaN,NaN,1980-01-01 00:00:00,Works on Paper,Ink and colour / Paper,1003,3/31/19,Sotheby's Hong Kong,94.0,179.0,1.0,0.0,0.0,1.0
3,Liang Guan,COWHERD,112090,64050.0,102480.0,NaN,NaN,1977-01-01 00:00:00,Works on Paper,Ink and colour / Paper,1004,3/31/19,Sotheby's Hong Kong,98.0,90.5,1.0,0.0,0.0,1.0
4,Yen-Yung Ting,THE EIGHT IMMORTALS,NaN,102480.0,204960.0,NaN,NaN,1975-01-01 00:00:00,Works on Paper,Ink and colour / Paper,1005,3/31/19,Sotheby's Hong Kong,137.0,70.0,1.0,0.0,0.0,1.0


### Category and medium

In [560]:
# all medium_clean and medium to lowercase
df_sothebys['category'] = df_sothebys['category'].str.lower()
df_sothebys['medium'] = df_sothebys['medium'].str.lower()

# in medium_clean remove "works on"
df_sothebys['category'] = df_sothebys['category'].str.replace('works on','')

# remove all white space
df_sothebys['category'] = df_sothebys['category'].str.strip()

In [561]:
df_sothebys.head()

,artist,title,sales_dollar,low_estimate,high_estimate,edition,foundry,created,category,medium,auction_lot,auction_date,auction_location,height,width,signed,stamped,inscribed,dated
0,Liang Guan,STILL LIFE,240190,25620.0,51240.0,NaN,NaN,NaT,paintings,oil / paperboard,1001,3/31/19,Sotheby's Hong Kong,28.0,33.0,1.0,0.0,0.0,0.0
1,Liang Guan,COMPOSITION,1252180,256200.0,512400.0,NaN,NaN,1948-01-01 00:00:00,paintings,oil / canvas,1002,3/31/19,Sotheby's Hong Kong,54.6,64.3,0.0,0.0,0.0,0.0
2,Liang Guan,MONKEY KING MAKING HAVOC IN HEAVEN,1713340,256200.0,512400.0,NaN,NaN,1980-01-01 00:00:00,paper,ink and colour / paper,1003,3/31/19,Sotheby's Hong Kong,94.0,179.0,1.0,0.0,0.0,1.0
3,Liang Guan,COWHERD,112090,64050.0,102480.0,NaN,NaN,1977-01-01 00:00:00,paper,ink and colour / paper,1004,3/31/19,Sotheby's Hong Kong,98.0,90.5,1.0,0.0,0.0,1.0
4,Yen-Yung Ting,THE EIGHT IMMORTALS,NaN,102480.0,204960.0,NaN,NaN,1975-01-01 00:00:00,paper,ink and colour / paper,1005,3/31/19,Sotheby's Hong Kong,137.0,70.0,1.0,0.0,0.0,1.0


### Auction date

In [562]:
# auction date to datetime
df_sothebys['auction_date'] = pd.to_datetime(df_sothebys['auction_date'], format='%d/%m/%Y', errors='ignore')

### Rename Sales_dollar

In [567]:
# rename sales_dollar to sales_price
df_sothebys.rename(columns={'sales_dollar':'sales_price'}, inplace=True)

In [568]:
df_sothebys.head()

,artist,title,sales_price,low_estimate,high_estimate,edition,foundry,created,category,medium,auction_lot,auction_date,auction_location,height,width,signed,stamped,inscribed,dated
0,Liang Guan,STILL LIFE,240190,25620.0,51240.0,NaN,NaN,NaT,paintings,oil / paperboard,1001,3/31/19,Sotheby's Hong Kong,28.0,33.0,1.0,0.0,0.0,0.0
1,Liang Guan,COMPOSITION,1252180,256200.0,512400.0,NaN,NaN,1948-01-01 00:00:00,paintings,oil / canvas,1002,3/31/19,Sotheby's Hong Kong,54.6,64.3,0.0,0.0,0.0,0.0
2,Liang Guan,MONKEY KING MAKING HAVOC IN HEAVEN,1713340,256200.0,512400.0,NaN,NaN,1980-01-01 00:00:00,paper,ink and colour / paper,1003,3/31/19,Sotheby's Hong Kong,94.0,179.0,1.0,0.0,0.0,1.0
3,Liang Guan,COWHERD,112090,64050.0,102480.0,NaN,NaN,1977-01-01 00:00:00,paper,ink and colour / paper,1004,3/31/19,Sotheby's Hong Kong,98.0,90.5,1.0,0.0,0.0,1.0
4,Yen-Yung Ting,THE EIGHT IMMORTALS,NaN,102480.0,204960.0,NaN,NaN,1975-01-01 00:00:00,paper,ink and colour / paper,1005,3/31/19,Sotheby's Hong Kong,137.0,70.0,1.0,0.0,0.0,1.0


## Merge both datasets into one

In [584]:
# order the columns of the dataframe alphabetically
df_sothebys = df_sothebys.reindex(sorted(df_sothebys.columns), axis=1)

df_sothebys.head()

,artist,auction_date,auction_location,auction_lot,category,created,dated,edition,foundry,height,high_estimate,inscribed,low_estimate,medium,sales_price,signed,stamped,title,width
0,Liang Guan,3/31/19,Sotheby's Hong Kong,1001,paintings,NaT,0.0,NaN,NaN,4.34,51240.0,0.0,25620.0,oil / paperboard,240190,1.0,0.0,STILL LIFE,5.12
1,Liang Guan,3/31/19,Sotheby's Hong Kong,1002,paintings,1948-01-01 00:00:00,0.0,NaN,NaN,8.46,512400.0,0.0,256200.0,oil / canvas,1252180,0.0,0.0,COMPOSITION,9.97
2,Liang Guan,3/31/19,Sotheby's Hong Kong,1003,paper,1980-01-01 00:00:00,1.0,NaN,NaN,14.57,512400.0,0.0,256200.0,ink and colour / paper,1713340,1.0,0.0,MONKEY KING MAKING HAVOC IN HEAVEN,27.75
3,Liang Guan,3/31/19,Sotheby's Hong Kong,1004,paper,1977-01-01 00:00:00,1.0,NaN,NaN,15.19,102480.0,0.0,64050.0,ink and colour / paper,112090,1.0,0.0,COWHERD,14.03
4,Yen-Yung Ting,3/31/19,Sotheby's Hong Kong,1005,paper,1975-01-01 00:00:00,1.0,NaN,NaN,21.24,204960.0,0.0,102480.0,ink and colour / paper,NaN,1.0,0.0,THE EIGHT IMMORTALS,10.85


In [585]:
df_sothebys.shape

(5180, 19)

In [586]:
# order the columns of the dataframe alphabetically
df_christies = df_christies.reindex(sorted(df_christies.columns), axis=1)

df_christies.head()

,artist,auction_date,auction_location,auction_lot,category,created,dated,edition,foundry,height,high_estimate,inscribed,low_estimate,medium,sales_price,signed,stamped,title,width
0,Binhong Huang,05/30/2017,Christie's Hong Kong,1364,paper,1947-01-01 00:00:00,0.0,NaN,NaN,33.66,64150.0,1.0,38490.0,watercolor and ink / paper,45900.0,1.0,0.0,Misty Landscape,16.93
1,Binhong Huang,05/30/2017,Christie's Hong Kong,1365,paper,1990-01-01 00:00:00,0.0,NaN,NaN,13.58,64150.0,1.0,38490.0,watercolor and ink / paper,NaN,1.0,0.0,Conversations in the Mountain,10.43
2,Yun Tang,05/30/2017,Christie's Hong Kong,1366,paper,NaT,0.0,NaN,NaN,NaN,23090.0,1.0,15400.0,watercolor and ink / paper,22950.0,1.0,0.0,Peony and Insects / Peony and Butterfly (2),NaN
3,Binhong Huang,05/30/2017,Christie's Hong Kong,1367,paper,NaT,0.0,NaN,NaN,40.94,449050.0,1.0,320750.0,watercolor and ink / paper,433330.0,1.0,0.0,Retreat in the Mountains,15.55
4,Yun Tang,05/30/2017,Christie's Hong Kong,1368,paper,1940-01-01 00:00:00,0.0,NaN,NaN,39.53,12830.0,1.0,10260.0,watercolor and ink / paper,12240.0,1.0,0.0,Villagers in the Woods,18.82


In [587]:
df_christies.shape

(7067, 19)

In [588]:
# merge the two dataframes vertically
df_HK = pd.concat([df_christies, df_sothebys], axis=0)

In [589]:
# add a curreny column with "USD" as value
df_HK['currency'] = 'USD'

In [597]:
df_HK["auction_location"][0].str.split("Hong Kong")

0    [Christie's , ]
0     [Sotheby's , ]
Name: auction_location, dtype: object

In [616]:
# create aution house column: if Sotheby's then Sotheby's else Christie's
df_HK['auction_house'] = df_HK['auction_location'].apply(lambda x: 'Sotheby\'s' if 'Sothebys' in x else 'Christies')

In [617]:
# auction_loation column becomes "Hong Kong"
df_HK['auction_location'] = 'Hong Kong'

In [618]:
# sort columns of  df_HK alphabetically
df_HK = df_HK.reindex(sorted(df_HK.columns), axis=1)

In [619]:
df_HK.shape

(12247, 21)

In [620]:
df_HK.head()

,artist,auction_date,auction_house,auction_location,auction_lot,category,created,currency,dated,edition,foundry,height,high_estimate,inscribed,low_estimate,medium,sales_price,signed,stamped,title,width
0,Binhong Huang,05/30/2017,Christies,Hong Kong,1364,paper,1947-01-01 00:00:00,USD,0.0,NaN,NaN,33.66,64150.0,1.0,38490.0,watercolor and ink / paper,45900.0,1.0,0.0,Misty Landscape,16.93
1,Binhong Huang,05/30/2017,Christies,Hong Kong,1365,paper,1990-01-01 00:00:00,USD,0.0,NaN,NaN,13.58,64150.0,1.0,38490.0,watercolor and ink / paper,NaN,1.0,0.0,Conversations in the Mountain,10.43
2,Yun Tang,05/30/2017,Christies,Hong Kong,1366,paper,NaT,USD,0.0,NaN,NaN,NaN,23090.0,1.0,15400.0,watercolor and ink / paper,22950.0,1.0,0.0,Peony and Insects / Peony and Butterfly (2),NaN
3,Binhong Huang,05/30/2017,Christies,Hong Kong,1367,paper,NaT,USD,0.0,NaN,NaN,40.94,449050.0,1.0,320750.0,watercolor and ink / paper,433330.0,1.0,0.0,Retreat in the Mountains,15.55
4,Yun Tang,05/30/2017,Christies,Hong Kong,1368,paper,1940-01-01 00:00:00,USD,0.0,NaN,NaN,39.53,12830.0,1.0,10260.0,watercolor and ink / paper,12240.0,1.0,0.0,Villagers in the Woods,18.82


## Lets clean the Artbank dataset

In [693]:
df_artbank = pd.read_csv('Data_raw/cs_stony_brook.csv')
df_artbank = pd.DataFrame(df_artbank)

In [694]:
# show all columns 
pd.set_option('display.max_columns', None)
df_artbank.head()

,name,country,birth,death,title,medium,size,signed,stamped,inscribed,dating,lot_num,auction_date,auction_house,lower_estimate,upper_estimate,actual_price,currency,with_prem,edition,foundry,impression
0,Abraham Rutgers,Netherlands,1632.0,1699.0,A STREET IN JISP ON A WINTER&#39;S DAY,Pen,7.60 x 12.05 in,Yes,No,No,NaN,29,07-09-2008,Sothebys - United Kingdom: London Bond Street,50000.0,70000.0,85250,GBP,Yes,NaN,NaN,NaN
1,Abraham Rutgers,Netherlands,1632.0,1699.0,"Skaters on the Vecht in winter, with a slantin...","Chalk pen and brown ink, brown and grey wash, ...",10.55 x 8.27 in,No,No,No,NaN,71,07-10-2014,Christies - United Kingdom: London King Street,50000.0,70000.0,302500,GBP,Yes,NaN,NaN,NaN
2,Ad Reinhardt,United States of America,1913.0,1967.0,"Abstract Painting, Red",Oil on canvas,30.12 x 30.00 in,Yes,No,No,1953.0,22,11-12-2013,Christies - The United States: New York ROCKEF...,1400000.0,1800000.0,2741000,USD,Yes,NaN,NaN,NaN
3,Ad Reinhardt,United States of America,1913.0,1967.0,Abstract Painting,Oil on canvas,14.00 x 14.00 in,Yes,No,No,1958.0,198,05-12-2011,Christies - The United States: New York ROCKEF...,30000.0,40000.0,116500,USD,Yes,NaN,NaN,NaN
4,Ad Reinhardt,United States of America,1913.0,1967.0,Abstract Painting,Oil and acrylic on canvas,78.00 x 24.00 in,Yes,No,No,1950.0,50,05-11-2011,Christies - The United States: New York ROCKEF...,2000000.0,3000000.0,2210500,USD,Yes,NaN,NaN,NaN


### Rename/Drop some columns

In [695]:
#  drop country, birth, death, with_prem and impression columns
df_artbank.drop(columns=['birth','death','with_prem','impression'],inplace=True)

A bit annoying to drop country column as it would be a nice metric but that's life ? Or we could only keep this second dataset but have less data ?

In [696]:
# rename name as artist
df_artbank.rename(columns={'name':'artist'}, inplace=True)
# renamelot_num as auction_lot
df_artbank.rename(columns={'lot_num':'auction_lot'}, inplace=True)
#  rename lower_estimate as low_estimate
df_artbank.rename(columns={'lower_estimate':'low_estimate'}, inplace=True)
#  rename upper_estimate as high_estimate
df_artbank.rename(columns={'upper_estimate':'high_estimate'}, inplace=True)
# rename actual_price as sales_price
df_artbank.rename(columns={'actual_price':'sales_price'}, inplace=True)

In [697]:
# rename column dating as created
df_artbank.rename(columns={'dating':'created'}, inplace=True)

In [698]:
# sort columns alphabetically
df_artbank = df_artbank.reindex(sorted(df_artbank.columns), axis=1)

In [699]:
df_artbank.head()

,artist,auction_date,auction_house,auction_lot,country,created,currency,edition,foundry,high_estimate,inscribed,low_estimate,medium,sales_price,signed,size,stamped,title
0,Abraham Rutgers,07-09-2008,Sothebys - United Kingdom: London Bond Street,29,Netherlands,NaN,GBP,NaN,NaN,70000.0,No,50000.0,Pen,85250,Yes,7.60 x 12.05 in,No,A STREET IN JISP ON A WINTER&#39;S DAY
1,Abraham Rutgers,07-10-2014,Christies - United Kingdom: London King Street,71,Netherlands,NaN,GBP,NaN,NaN,70000.0,No,50000.0,"Chalk pen and brown ink, brown and grey wash, ...",302500,No,10.55 x 8.27 in,No,"Skaters on the Vecht in winter, with a slantin..."
2,Ad Reinhardt,11-12-2013,Christies - The United States: New York ROCKEF...,22,United States of America,1953.0,USD,NaN,NaN,1800000.0,No,1400000.0,Oil on canvas,2741000,Yes,30.12 x 30.00 in,No,"Abstract Painting, Red"
3,Ad Reinhardt,05-12-2011,Christies - The United States: New York ROCKEF...,198,United States of America,1958.0,USD,NaN,NaN,40000.0,No,30000.0,Oil on canvas,116500,Yes,14.00 x 14.00 in,No,Abstract Painting
4,Ad Reinhardt,05-11-2011,Christies - The United States: New York ROCKEF...,50,United States of America,1950.0,USD,NaN,NaN,3000000.0,No,2000000.0,Oil and acrylic on canvas,2210500,Yes,78.00 x 24.00 in,No,Abstract Painting


### Auction date

In [700]:
# change format from xx-xx-xxxx to xx/xx/xxxx
df_artbank['auction_date'] = df_artbank['auction_date'].str.replace('-','/')

In [701]:
# auction_date as datetime
df_artbank['auction_date'] = pd.to_datetime(df_artbank['auction_date'], format='%d/%m/%Y', errors='ignore')

### Dating

In [702]:
# create empty column dating
df_artbank['dated'] = np.nan

In [703]:
# if created is nan dating is 0 else 1
df_artbank['dated'] = df_artbank['created'].apply(lambda x: 0 if pd.isnull(x) else 1)

### Created

In [704]:
#  if not nan, convert created to timestamp (year)
df_artbank['created'] = df_artbank['created'].apply(lambda x: pd.to_datetime(x, format='%Y', errors='ignore') if pd.notnull(x) else x)

### Inscribed, Signed and Stamped

In [705]:
# function to replace Yes with 1 and No with 0
def yes_no(x):
    if x == 'Yes':
        return 1
    elif x == 'No':
        return 0
    else:
        return x

In [706]:
# applu yes_no function to signed, stamped, inscribed columns
df_artbank['signed'] = df_artbank['signed'].apply(yes_no)
df_artbank['stamped'] = df_artbank['stamped'].apply(yes_no)
df_artbank['inscribed'] = df_artbank['inscribed'].apply(yes_no)

In [707]:
# sort columns alphabetically
df_artbank = df_artbank.reindex(sorted(df_artbank.columns), axis=1)

### Auction house and location

In [708]:
df_artbank['auction_house'].str.split('-',expand=True)

,0,1,2,3,4,5,6,7,8
0,Sothebys,United Kingdom: London Bond Street,None,None,None,None,None,None,None
1,Christies,United Kingdom: London King Street,None,None,None,None,None,None,None
2,Christies,The United States: New York ROCKEFELLER PLAZA,None,None,None,None,None,None,None
3,Christies,The United States: New York ROCKEFELLER PLAZA,None,None,None,None,None,None,None
4,Christies,The United States: New York ROCKEFELLER PLAZA,None,None,None,None,None,None,None
...,...,...,...,...,...,...,...,...,...
41608,Christies,China: Hong Kong,None,None,None,None,None,None,None
41609,Christies,China: Hong Kong,None,None,None,None,None,None,None
41610,Sothebys,China: Hong Kong,None,None,None,None,None,None,None
41611,Sothebys,China: Hong Kong,None,None,None,None,None,None,None


In [709]:
# split auction_house on - and take the first elementfor auction_house column and the second element for auction_location column
df_artbank[['auction_house','auction_location']] = df_artbank['auction_house'].str.split('-',expand=True)[[0,1]]

### Size

In [710]:
# split size column into height and width
df_artbank['height'] = df_artbank['size'].str.split('x').str[0]
df_artbank['width'] = df_artbank['size'].str.split('x').str[1].str.split('in').str[0]

# remove white space
df_artbank['height'] = df_artbank['height'].str.strip()
df_artbank['width'] = df_artbank['width'].str.strip()

# convert height and width to float
df_artbank['height'] = df_artbank['height'].astype(float)
df_artbank['width'] = df_artbank['width'].astype(float)

# drop size column
df_artbank.drop(columns=['size'],inplace=True)

In [711]:
# sort columns alphabetically
df_artbank = df_artbank.reindex(sorted(df_artbank.columns), axis=1)

### Medium

WE WOULD NEED SOME MORE PREPROCESSING HERE

In [712]:
# unique medium value 
len(df_artbank['medium'].unique())

6988

In [713]:
df_artbank.head()

,artist,auction_date,auction_house,auction_location,auction_lot,country,created,currency,dated,edition,foundry,height,high_estimate,inscribed,low_estimate,medium,sales_price,signed,stamped,title,width
0,Abraham Rutgers,07/09/2008,Sothebys,United Kingdom: London Bond Street,29,Netherlands,NaN,GBP,0,NaN,NaN,7.60,70000.0,0,50000.0,Pen,85250,1,0,A STREET IN JISP ON A WINTER&#39;S DAY,12.05
1,Abraham Rutgers,07/10/2014,Christies,United Kingdom: London King Street,71,Netherlands,NaN,GBP,0,NaN,NaN,10.55,70000.0,0,50000.0,"Chalk pen and brown ink, brown and grey wash, ...",302500,0,0,"Skaters on the Vecht in winter, with a slantin...",8.27
2,Ad Reinhardt,11/12/2013,Christies,The United States: New York ROCKEFELLER PLAZA,22,United States of America,1953-01-01 00:00:00,USD,1,NaN,NaN,30.12,1800000.0,0,1400000.0,Oil on canvas,2741000,1,0,"Abstract Painting, Red",30.00
3,Ad Reinhardt,05/12/2011,Christies,The United States: New York ROCKEFELLER PLAZA,198,United States of America,1958-01-01 00:00:00,USD,1,NaN,NaN,14.00,40000.0,0,30000.0,Oil on canvas,116500,1,0,Abstract Painting,14.00
4,Ad Reinhardt,05/11/2011,Christies,The United States: New York ROCKEFELLER PLAZA,50,United States of America,1950-01-01 00:00:00,USD,1,NaN,NaN,78.00,3000000.0,0,2000000.0,Oil and acrylic on canvas,2210500,1,0,Abstract Painting,24.00


In [714]:
df_HK.head()

,artist,auction_date,auction_house,auction_location,auction_lot,category,created,currency,dated,edition,foundry,height,high_estimate,inscribed,low_estimate,medium,sales_price,signed,stamped,title,width
0,Binhong Huang,05/30/2017,Christies,Hong Kong,1364,paper,1947-01-01 00:00:00,USD,0.0,NaN,NaN,33.66,64150.0,1.0,38490.0,watercolor and ink / paper,45900.0,1.0,0.0,Misty Landscape,16.93
1,Binhong Huang,05/30/2017,Christies,Hong Kong,1365,paper,1990-01-01 00:00:00,USD,0.0,NaN,NaN,13.58,64150.0,1.0,38490.0,watercolor and ink / paper,NaN,1.0,0.0,Conversations in the Mountain,10.43
2,Yun Tang,05/30/2017,Christies,Hong Kong,1366,paper,NaT,USD,0.0,NaN,NaN,NaN,23090.0,1.0,15400.0,watercolor and ink / paper,22950.0,1.0,0.0,Peony and Insects / Peony and Butterfly (2),NaN
3,Binhong Huang,05/30/2017,Christies,Hong Kong,1367,paper,NaT,USD,0.0,NaN,NaN,40.94,449050.0,1.0,320750.0,watercolor and ink / paper,433330.0,1.0,0.0,Retreat in the Mountains,15.55
4,Yun Tang,05/30/2017,Christies,Hong Kong,1368,paper,1940-01-01 00:00:00,USD,0.0,NaN,NaN,39.53,12830.0,1.0,10260.0,watercolor and ink / paper,12240.0,1.0,0.0,Villagers in the Woods,18.82


## Merge both datasets

In [715]:
# add a column of Nan in df_artbank called category
df_artbank['category'] = np.nan

# add a column of Nan in df_HK called country
df_HK['country'] = np.nan

In [716]:
# sort both dataframes alphabetically
df_HK = df_HK.reindex(sorted(df_HK.columns), axis=1)
df_artbank = df_artbank.reindex(sorted(df_artbank.columns), axis=1)

In [717]:
# merge the two dataframes vertically
df = pd.concat([df_HK, df_artbank], axis=0)

In [719]:
df.head()

,artist,auction_date,auction_house,auction_location,auction_lot,category,country,created,currency,dated,edition,foundry,height,high_estimate,inscribed,low_estimate,medium,sales_price,signed,stamped,title,width
0,Binhong Huang,05/30/2017,Christies,Hong Kong,1364,paper,NaN,1947-01-01 00:00:00,USD,0.0,NaN,NaN,33.66,64150.0,1.0,38490.0,watercolor and ink / paper,45900.0,1.0,0.0,Misty Landscape,16.93
1,Binhong Huang,05/30/2017,Christies,Hong Kong,1365,paper,NaN,1990-01-01 00:00:00,USD,0.0,NaN,NaN,13.58,64150.0,1.0,38490.0,watercolor and ink / paper,NaN,1.0,0.0,Conversations in the Mountain,10.43
2,Yun Tang,05/30/2017,Christies,Hong Kong,1366,paper,NaN,NaT,USD,0.0,NaN,NaN,NaN,23090.0,1.0,15400.0,watercolor and ink / paper,22950.0,1.0,0.0,Peony and Insects / Peony and Butterfly (2),NaN
3,Binhong Huang,05/30/2017,Christies,Hong Kong,1367,paper,NaN,NaT,USD,0.0,NaN,NaN,40.94,449050.0,1.0,320750.0,watercolor and ink / paper,433330.0,1.0,0.0,Retreat in the Mountains,15.55
4,Yun Tang,05/30/2017,Christies,Hong Kong,1368,paper,NaN,1940-01-01 00:00:00,USD,0.0,NaN,NaN,39.53,12830.0,1.0,10260.0,watercolor and ink / paper,12240.0,1.0,0.0,Villagers in the Woods,18.82


In [721]:
df.shape

(53860, 22)

In [722]:
# export to csv
df.to_csv('cleaned_data.csv', index=False)